                                     NLP Preprocessing
Since the Data set is very big,  we are unable to get the subset of the 900MB data set. We ran it for 3hrs still we are unable to get the subset of the data. 

So we have done preprocessing steps on 5MB data set provided. So Approach will be same, once we have finished complete analysis, we can extend the code to 50MB data set.

Steps involved in Yelp data Set challenge:

Approach1: 

    1.  Get the Data.
    2.  Tokenizing, Parsing, Converting text to lower case, Stopword removal, Stemming.
    3.  Convert the text in the form of tf-idf matrix
    4.  Apply K-means clustering Algorithm
    5.  Divide into 5 different clusters and identify the unique words belongs to each cluster.
    6.  Caluclate the sentiment Analysis for each review
    7.  Categorise the data grouped by "Business-ID" i.e. resturant-ID
    8.  Take each review at a time and identify whether it belongs to First Cluster or not., repeated for all the reviews.
    9.  Step-8 is repeated for all the clusters individally.
    10. Now Using the result from sentiment Analysis and reviews-classification(if first review belongs to 1st and 3rd 
        cluster then the ouput will look like "10100")., we categorise the Sentiment Distribution.
Approach2: 

    1.  Get the Data.
    2.  Tokenizing, Parsing, Converting text to lower case, Stopword removal, Stemming.
    3.  Convert the text in the form of tf-idf matrix
    4.  Apply K-means clustering Algorithm
    5.  Divide into 5 different clusters and identify the unique words belongs to each cluster.
    6.  Categorise the data grouped by "Business-ID" i.e. resturant-ID
    7.  Take each review and extract the sentence and identify the sentence belongs to which cluster and apply sentiment 
        analysis.
    8.  Apply step-7 to all the reviews. Then we can get the Sentiment Distribution.
    
 Team Members: Ajith vajrala, Basant Gupta

In [1]:
import pandas as pd
import pickle

In [2]:
with open('E:\Aegis\NLP\Project/yelp_data_subset.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
dtest = data
#Converting list to array
import numpy as np
myarray = np.asarray(dtest)

In [5]:
#Converting list to data frame using Pandas

df = pd.DataFrame(dtest, columns=['votes', 'user_id', 'review_id', 'text','business_id','date','type'])

In [6]:
df.head()

,votes,user_id,review_id,text,business_id,date,type
0,"{u'funny': 0, u'useful': 0, u'cool': 0}",PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review
1,"{u'funny': 0, u'useful': 0, u'cool': 0}",Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review
2,"{u'funny': 1, u'useful': 1, u'cool': 0}",auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Yes this place is a little out dated and not o...,5UmKMjUEUNdYWqANhGckJw,2015-10-31,review
3,"{u'funny': 0, u'useful': 0, u'cool': 0}",qiczib2fO_1VBG8IoCGvVg,pVMIt0a_QsKtuDfWVfSk2A,PROS: Italian hoagie was delicious. Friendly ...,5UmKMjUEUNdYWqANhGckJw,2015-12-26,review
4,"{u'funny': 0, u'useful': 1, u'cool': 0}",qEE5EvV-f-s7yHC0Z4ydJQ,AEyiQ_Y44isJmNbMTyoMKQ,First the only reason this place could possibl...,5UmKMjUEUNdYWqANhGckJw,2016-04-08,review


In [7]:
df.shape

(5503, 7)

In [8]:
#Data extraction is completed.
#
#Check for missing values in the data
#
df.apply(lambda x: sum(x.isnull()))

votes          0
user_id        0
review_id      0
text           0
business_id    0
date           0
type           0
dtype: int64

In [9]:
#There are no missing values in the data

In [10]:
df_ = df
del df_['votes']

#Look at no.of categorical variables in the data
df_.apply(lambda x: len(x.unique()))

user_id        3519
review_id      5503
text           5499
business_id     311
date           2185
type              1
dtype: int64

In [12]:
import nltk
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [13]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
#
#
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [14]:
Reviews_text = df_['text']

In [16]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
import re
#
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Reviews_text:
    allwords_stemmed = tokenize_and_stem(i) #
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)